<a href="https://www.kaggle.com/code/sravanneeli/convert-train-and-test-multiple-parquet-files?scriptVersionId=97099891" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [2]:
import dask.dataframe as dd
import gc

In [3]:
test_df = pd.read_csv('../input/amex-default-prediction/test_data.csv', nrows=10)

In [4]:
dtype_dict = {}

In [5]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
for col in cat_cols:
    if test_df[col].dtype == "float64":
        dtype_dict[col] = "int8" # category to 
    else:
        dtype_dict[col] = str

In [6]:
for col in test_df.columns:
    
    if test_df[col].dtype == "float64":
        dtype_dict[col] = "float32"

In [7]:
train_df = pd.read_csv('../input/amex-default-prediction/train_data.csv', chunksize=100000)

In [8]:
for index, row in enumerate(train_df):
    row = row.astype(dtype_dict)
    row['customer_ID'] = row['customer_ID'].astype('|S')
    row['S_2'] = row['S_2'].astype('datetime64[ns]')
    row.to_parquet(f'train_file_{index+1}.pqt', index=False)
    del(row)
    gc.collect()

In [9]:
del(train_df)

In [10]:
gc.collect()

42

In [11]:
test_df = pd.read_csv('../input/amex-default-prediction/test_data.csv', chunksize=100000)

In [12]:
for index, row in enumerate(test_df):
    row = row.astype(dtype_dict)
    row['customer_ID'] = row['customer_ID'].astype('|S')
    row['S_2'] = row['S_2'].astype('datetime64[ns]')
    row.to_parquet(f'test_file_{index+1}.pqt', index=False)
    del(row)
    gc.collect()

In [13]:
del(test_df)
gc.collect()

21